In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
from make_weights2 import make_weights2, concat_weights, get_stats_df
from visualize import display_saliency


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ouren\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ouren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ouren\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ouren\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ouren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ouren\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\User

# Load Full List
New Synonym files will be CSV

In [ ]:
base_path = Path('C:/Users/ouren/Documents/School_Local/eHDV_Adoption/LDA_Processing/notebooks')
syn_path = base_path / 'synonyms3.xlsx'

syn_df = pd.read_excel(syn_path,sheet_name=1)

# Randomly Select 9 Synonyms per Category

In [ ]:
key = ['long_term_cost','short_term_cost','policy_incentives',
        'suitability_compatibility','familiarity_knowledge','norms_attitudes',
        'brand_image','sustainability','reliability_uncertainty','driver_acceptance']

In [ ]:
def get_random_syns(df,synonym):
    term_dict = {'word (in order of signifigance)':[],'synonym':[]}
    # cut df to synonym target
    tmp_df = df.loc[df['synonym'] == synonym]
    tmp_df.reset_index(inplace=True)
    # select random list to be used as indicies
    index_list = np.random.choice(tmp_df['synonym'].size , 9 , replace=False)
    for num in index_list:
        word = tmp_df['word (in order of signifigance)'].iloc[num]
        term_dict['word (in order of signifigance)'].append(word)
        term_dict['synonym'].append(synonym)
    # randomly choose 9 of the provided words to represent the category
    # for i in range(9):
    #     word = np.random.choice(tmp_df['word (in order of signifigance)'],replace=False)
    #     term_dict['word (in order of signifigance)'].append(word)
    #     term_dict['synonym'].append(term)
    # return df 
    return pd.DataFrame(term_dict)
    

The block below chooses random words from overall pool to represent each synonym
as many times as specified in outer loop

In [ ]:
# run get_random_syns for all synonyms (terms)
new_syn_df = pd.DataFrame(columns = ['word (in order of signifigance)','synonym'])
k = 0
for i in range(100):
    new_syn_df = pd.DataFrame(columns = ['word (in order of signifigance)','synonym'])
    for synonym in key:
        word_syn_df = get_random_syns(syn_df,synonym)
        set = [new_syn_df,word_syn_df]
        new_syn_df = pd.concat(set)
    new_syn_df.to_csv(base_path / 'Syn_Rules' / 'syn_rules_{}.csv'.format(k))
    k += 1

# Apply New Synonym Rules
Scripts regarding this test are marked with a 2 suffix (preprocess2,LDA2,makeweights2)

### Feed each set of rules to preprocess2
syn_path is now a variable of preprocess 2 that can be fed from weightmaker2 to lda2 to finally arrive at preprocess2


In [2]:
def get_filepaths(folder_path):
   
        file_paths = []  # List which will store all of the full filepaths.

        file_location = folder_path
        # Walk the tree.
        for root, directories, files in os.walk(file_location):
            for filename in files:
                # Join the two strings in order to form the full filepath.
                filepath = os.path.join(root, filename)
                file_paths.append(filepath)  # Add it to the list.

        return file_paths 

In [3]:
BASE_PATH = 'C:/Users/ouren/Documents/School_Local/eHDV_Adoption/LDA_Processing/notebooks/'
DIRECTORY = 'Papers'
TARGET_PATH = 'C:/Users/ouren/Documents/School_Local/eHDV_Adoption/LDA_Processing/notebooks/Outputs'
INDUSTRY_DIRECTORY = 'industry_articles'

ACADEMIC_FILE = '3_31_academic'
INDUSTRY_FILE = '3_31_industry'

ITERATIONS = 1

In [4]:
SYN_FOLDER = BASE_PATH + 'Syn_Rules'
syn_paths = get_filepaths(SYN_FOLDER)

### Build models with different synonym rules

In [6]:
for path in syn_paths:
    df = pd.read_csv(path)
    if df['synonym'].size < 90:
        print(path)

In [ ]:
master_df = pd.DataFrame(columns = ['Term',
                                    'Academic_Normalized_Saliency',
                                    'Industry_Normalized_Saliency',
                                    'Syn_Rules'])
k = 0
for path in syn_paths:
    # create make_weights objects for both corpora
    weight_maker = make_weights2(base_path = BASE_PATH,
                            directory = DIRECTORY,
                            target_path = TARGET_PATH,
                            iterations = ITERATIONS,
                            name = ACADEMIC_FILE,
                            save = False,
                            syn_path = path)

    print('Base Path: {}\nDirectory: {}\nTarget Path: {}'.format(BASE_PATH,DIRECTORY,TARGET_PATH))

    industry_weight_maker = make_weights2(base_path = BASE_PATH,
                            directory = INDUSTRY_DIRECTORY,
                            target_path = TARGET_PATH,
                            iterations = ITERATIONS,
                            name = INDUSTRY_FILE,
                            save = False,
                            syn_path = path)
    
    # get weights with given synonym rules
    weight_maker.create_weights()
    industry_weight_maker.create_weights()

    academic_weights_df = weight_maker.final_saliency_df
    industry_weights_df = industry_weight_maker.final_saliency_df

    # combine them into one dataframe
    compare_df = concat_weights(academic_weights_df,industry_weights_df)
    compare_df['Syn_Rules'] = path

    # log each iteration
    set = [master_df,compare_df]
    master_df = pd.concat(set)

    if k % 5 == 0:
        master_df.to_csv('./Outputs/3_31_syn_verification_raw.csv')
    else:
        continue

master_df.to_csv('./Outputs/3_31_syn_verification_raw.csv')



In [7]:
master_df.to_csv('./Outputs/3_31_syn_verification_raw.csv')

# Synonym Experiment Analysis

In [2]:
base_path = Path('C:/Users/ouren/Documents/School_Local/eHDV_Adoption/LDA_Processing/notebooks')
syn_test_results_path = base_path / 'Outputs/3_31_syn_verification_raw.csv'

In [3]:
df = pd.read_csv(syn_test_results_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    300 non-null    int64  
 1   Term                          300 non-null    object 
 2   Academic_Normalized_Saliency  297 non-null    float64
 3   Industry_Normalized_Saliency  283 non-null    float64
 4   Syn_Rules                     300 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 11.8+ KB


In [5]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 280 entries, 0 to 299
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    280 non-null    int64  
 1   Term                          280 non-null    object 
 2   Academic_Normalized_Saliency  280 non-null    float64
 3   Industry_Normalized_Saliency  280 non-null    float64
 4   Syn_Rules                     280 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 13.1+ KB


In [6]:
# modify for get_stats_df()
dfa = df[['Term','Academic_Normalized_Saliency']].copy()
dfi = df[['Term','Industry_Normalized_Saliency']].copy()

dfi = dfi.rename(columns = {
    "Industry_Normalized_Saliency" : "Normalized_Saliency"
})

dfa = dfa.rename(columns = {
    "Academic_Normalized_Saliency" : "Normalized_Saliency"
})

dfa_stats, dfi_stats = get_stats_df(dfa,dfi)

dfa_stats['Corpus'] = 'academic'
dfi_stats['Corpus'] = 'industry'

set = [dfa_stats,dfi_stats]

stats_df = pd.concat(set)

In [7]:
syn_tests = display_saliency(
    academic_stats_df = dfa_stats,
    industry_stats_df = dfi_stats,
    academic_color = '#5ebfbc',
    industry_color = '#88d184'
)

### Bar Chart With Error

In [8]:
syn_tests.plotly_barchart()
syn_tests.bc_fig

In [10]:
syn_tests.save_fig(fig=syn_tests.bc_fig,file_name = '3_31_barchart_with_error')

## Radar Plot

In [9]:
syn_tests.plotly_radar()
syn_tests.rad_fig

In [10]:
syn_tests.save_fig(fig=syn_tests.rad_fig,file_name = '3_31_radar')